# 神经网络模型：

In [76]:
import torch

class Para:
    # tensor_board_log_dir = 'runs/exp0'
    feature_column_start_name = 'ep_ratio_ttm'
    feature_column_end_name = 'BR'

    # 模型设置
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    classification = 2 # 2, 3

    # 权重
    cross_weight = list()
    if classification == 3:
        cross_weight = [1.0, 1.0 ,1.0]
    elif classification == 2:
        cross_weight = [1.0, 1.0]
    elif classification == 5:
        cross_weight = [1.0, 1.0, 1.0, 1.0, 1.0]

    batch_size = 16
    lr = 1e-3
    drop = 0.5
    epochs = 5

    # 数据集设置
    month_in_sample = range(0, 1)
    # month_test = range(36, 48)

    percent_cv = 0.1 # 10% cross validation

    data_path = 'data/zj_space_1d_rate_20d_17-21_pre'


    seed = 2022
    torch.manual_seed(seed)

    info_str0 = data_path[5:7]+'_'+'c'+str(classification)+'_s'+str(percent_cv)
    info_str1 = '_b'+str(batch_size)+'_lr'+str(lr)+'_d'+str(drop)+'_e'+str(epochs)
    info_str = info_str0 + info_str1

    save_model_path = 'models/'+'model_'+info_str+'.pth'

para = Para()
print(para.info_str)
print(para.save_model_path)
print(torch.cuda.is_available())
print(torch.cuda.device_count())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
    print(torch.cuda.current_device())

zj_c2_s0.1_b16_lr0.001_d0.5_e5
models/model_zj_c2_s0.1_b16_lr0.001_d0.5_e5.pth
True
1
GeForce MX150
0


### 构建训练集

In [77]:
import numpy as np
import pandas as pd

data_in_sample = None
for i_month in para.month_in_sample:
    file_name = para.data_path + '/' + str(i_month) + '.csv'
    data_curr_month = pd.read_csv(file_name)

    data_curr_month = data_curr_month.dropna(axis=0)

    data_curr_month.insert(loc=0, column='return_bin', value=np.nan)

    data_curr_month.loc[data_curr_month['yield_rate']>0, 'return_bin'] = 0
    data_curr_month.loc[data_curr_month['yield_rate']<=0, 'return_bin'] = 1

    if i_month == para.month_in_sample[0]:
        data_in_sample = data_curr_month
    else:
        data_in_sample = pd.concat([data_in_sample, data_curr_month])
        # data_in_sample = data_in_sample.append(data_curr_month)

data_in_sample

,return_bin,order_book_id,board_type,sector_code,month,date,yield_rate,ep_ratio_ttm,pb_ratio_ttm,sp_ratio_ttm,...,RSI10,SY,BIAS20,VOL30,VOL60,VOL120,VOLT20,VOLT60,AR,BR
0,1.0,600895.XSHG,MainBoard,Industrials,0,2017-01-03,-0.114933,-0.791238,1.432009,2.812825,...,-0.011242,0.549024,-0.041903,-0.564230,-0.549325,-0.271267,-0.795472,-0.740978,-0.529566,-0.355811
1,1.0,600895.XSHG,MainBoard,Industrials,1,2017-01-04,-0.142703,-0.796248,1.465440,2.781002,...,0.607237,0.549024,0.055605,-0.576310,-0.552245,-0.274636,-0.826546,-0.736372,-0.237217,-0.156750
2,1.0,600895.XSHG,MainBoard,Industrials,2,2017-01-05,-0.085969,-0.796802,1.469155,2.777486,...,0.418334,0.549024,0.084119,-0.584643,-0.550836,-0.277450,-0.882262,-0.731683,-0.311444,-0.186746
3,0.0,600895.XSHG,MainBoard,Industrials,3,2017-01-06,0.003819,-0.789557,1.420865,2.823504,...,-0.729904,0.549024,-0.008698,-0.590887,-0.552451,-0.289335,-0.951495,-0.725460,-0.070957,-0.042307
4,1.0,600895.XSHG,MainBoard,Industrials,4,2017-01-09,-0.081338,-0.795694,1.461726,2.784522,...,0.374730,1.313049,0.109629,-0.607630,-0.551179,-0.297392,-1.040746,-0.721606,0.040297,-0.024960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189,0.0,600895.XSHG,MainBoard,Industrials,1192,2021-11-29,0.165186,2.047309,-1.456029,1.358008,...,-1.092140,-0.215002,-0.245321,-0.642811,-0.558218,-0.371645,-0.954208,-0.016711,-1.393978,-1.555949
1190,0.0,600895.XSHG,MainBoard,Industrials,1193,2021-11-30,0.112782,2.018929,-1.428285,1.321979,...,-0.453417,-0.215002,-0.119088,-0.641637,-0.563386,-0.412730,-0.945411,0.004912,-1.501683,-1.416865
1191,0.0,600895.XSHG,MainBoard,Industrials,1194,2021-12-01,0.084123,2.008713,-1.418197,1.309009,...,-0.485559,0.549024,-0.065383,-0.641369,-0.565753,-0.455013,-0.944476,0.024277,-1.377001,-1.410093
1192,0.0,600895.XSHG,MainBoard,Industrials,1195,2021-12-02,0.130370,2.034357,-1.443418,1.341565,...,-0.214241,-0.215002,-0.156535,-0.639549,-0.582325,-0.471130,-0.935971,0.037707,-1.233049,-1.308074


In [78]:
X_in_sample = data_in_sample.loc[:, para.feature_column_start_name: para.feature_column_end_name]
y_in_sample = data_in_sample.loc[:, 'return_bin']

from sklearn.model_selection import train_test_split

X_train, X_cv, y_train, y_cv = train_test_split(X_in_sample, y_in_sample, test_size=para.percent_cv, shuffle=False) # True, random_state=para.seed)
# X_train, X_cv, y_train, y_cv = train_test_split(X_in_sample, y_in_sample, test_size=para.percent_cv, shuffle=False)

X_train

,ep_ratio_ttm,pb_ratio_ttm,sp_ratio_ttm,MACD_DIFF,MACD_DEA,MACD_HIST,RSI10,SY,BIAS20,VOL30,VOL60,VOL120,VOLT20,VOLT60,AR,BR
0,-0.791238,1.432009,2.812825,-0.370625,-0.445848,0.161384,-0.011242,0.549024,-0.041903,-0.564230,-0.549325,-0.271267,-0.795472,-0.740978,-0.529566,-0.355811
1,-0.796248,1.465440,2.781002,-0.332546,-0.427616,0.224682,0.607237,0.549024,0.055605,-0.576310,-0.552245,-0.274636,-0.826546,-0.736372,-0.237217,-0.156750
2,-0.796802,1.469155,2.777486,-0.296543,-0.405518,0.270218,0.418334,0.549024,0.084119,-0.584643,-0.550836,-0.277450,-0.882262,-0.731683,-0.311444,-0.186746
3,-0.789557,1.420865,2.823504,-0.287260,-0.385903,0.240978,-0.729904,0.549024,-0.008698,-0.590887,-0.552451,-0.289335,-0.951495,-0.725460,-0.070957,-0.042307
4,-0.795694,1.461726,2.784522,-0.256703,-0.363834,0.269870,0.374730,1.313049,0.109629,-0.607630,-0.551179,-0.297392,-1.040746,-0.721606,0.040297,-0.024960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1069,1.882620,-0.069050,0.381447,1.090411,1.082809,0.148608,0.803148,1.313049,0.479565,-0.167304,-0.413186,-0.480671,0.414797,-0.144454,0.734146,0.316231
1070,1.548900,0.441940,0.027573,1.355354,1.147511,0.772026,1.579800,2.077075,1.978701,-0.054721,-0.349578,-0.444601,0.684860,-0.033462,1.661760,1.173297
1071,1.764803,0.100384,0.256514,1.324192,1.192771,0.543023,0.693033,1.313049,0.753909,0.056861,-0.285012,-0.405725,0.523496,0.002012,1.109373,0.743855
1072,1.867228,-0.047534,0.365125,1.188336,1.200630,0.102511,0.226868,0.549024,0.175303,0.099657,-0.262937,-0.391219,0.187746,0.009831,0.903161,0.659904


In [79]:
y_train

0       1.0
1       1.0
2       1.0
3       0.0
4       1.0
       ... 
1069    1.0
1070    1.0
1071    1.0
1072    1.0
1073    1.0
Name: return_bin, Length: 1074, dtype: float64

In [80]:
data_train = pd.concat([X_train, y_train], axis=1)
data_cv = pd.concat([X_cv, y_cv], axis=1)
data_train

,ep_ratio_ttm,pb_ratio_ttm,sp_ratio_ttm,MACD_DIFF,MACD_DEA,MACD_HIST,RSI10,SY,BIAS20,VOL30,VOL60,VOL120,VOLT20,VOLT60,AR,BR,return_bin
0,-0.791238,1.432009,2.812825,-0.370625,-0.445848,0.161384,-0.011242,0.549024,-0.041903,-0.564230,-0.549325,-0.271267,-0.795472,-0.740978,-0.529566,-0.355811,1.0
1,-0.796248,1.465440,2.781002,-0.332546,-0.427616,0.224682,0.607237,0.549024,0.055605,-0.576310,-0.552245,-0.274636,-0.826546,-0.736372,-0.237217,-0.156750,1.0
2,-0.796802,1.469155,2.777486,-0.296543,-0.405518,0.270218,0.418334,0.549024,0.084119,-0.584643,-0.550836,-0.277450,-0.882262,-0.731683,-0.311444,-0.186746,1.0
3,-0.789557,1.420865,2.823504,-0.287260,-0.385903,0.240978,-0.729904,0.549024,-0.008698,-0.590887,-0.552451,-0.289335,-0.951495,-0.725460,-0.070957,-0.042307,0.0
4,-0.795694,1.461726,2.784522,-0.256703,-0.363834,0.269870,0.374730,1.313049,0.109629,-0.607630,-0.551179,-0.297392,-1.040746,-0.721606,0.040297,-0.024960,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1069,1.882620,-0.069050,0.381447,1.090411,1.082809,0.148608,0.803148,1.313049,0.479565,-0.167304,-0.413186,-0.480671,0.414797,-0.144454,0.734146,0.316231,1.0
1070,1.548900,0.441940,0.027573,1.355354,1.147511,0.772026,1.579800,2.077075,1.978701,-0.054721,-0.349578,-0.444601,0.684860,-0.033462,1.661760,1.173297,1.0
1071,1.764803,0.100384,0.256514,1.324192,1.192771,0.543023,0.693033,1.313049,0.753909,0.056861,-0.285012,-0.405725,0.523496,0.002012,1.109373,0.743855,1.0
1072,1.867228,-0.047534,0.365125,1.188336,1.200630,0.102511,0.226868,0.549024,0.175303,0.099657,-0.262937,-0.391219,0.187746,0.009831,0.903161,0.659904,1.0


In [81]:
from torch.utils.data import TensorDataset
import torch

X_train_ndarray = X_train.values
y_train_ndarray = y_train.values

train_dataset = TensorDataset(torch.from_numpy(X_train_ndarray).type(torch.FloatTensor), torch.from_numpy(y_train_ndarray).type(torch.LongTensor))

# for X_train_temp, y_train_temp in train_dataset:
#     print(X_train_temp, y_train_temp)
#     print(X_train_temp.dtype, y_train_temp.dtype)

In [82]:
X_cv_ndarray = X_cv.values
y_cv_ndarray = y_cv.values

cv_dataset = TensorDataset(torch.from_numpy(X_cv_ndarray).type(torch.FloatTensor), torch.from_numpy(y_cv.values).type(torch.LongTensor))

# for X_cv_temp, y_cv_temp in cv_dataset:
#     print(X_cv_temp, y_cv_temp)
#     print(X_cv_temp.dtype, y_cv_temp.dtype)

In [83]:
from torch.utils.data import DataLoader


train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=para.batch_size,
    shuffle=True
)

cv_dataloader = DataLoader(
    dataset=cv_dataset,
    batch_size=para.batch_size,
    shuffle=True
)

### 构建测试集

In [84]:
# data_test = None
# for i_month in para.month_test:
#
#     file_name = para.data_path + '/' + str(i_month) + '.csv'
#     data_curr_month = pd.read_csv(file_name)
#
#     data_curr_month = data_curr_month.dropna(axis=0)
#
#     data_curr_month = label_data(data=data_curr_month, percent_select=para.percent_select)
#
#     if i_month == para.month_test[0]:
#         data_test = data_curr_month
#     else:
#         data_test = pd.concat([data_test, data_curr_month])
#         # data_test = data_test.append(data_curr_month)
#
# X_test = data_test.loc[:, para.feature_column_start_name: para.feature_column_end_name]
# y_test = data_test.loc[:, 'return_bin']

In [85]:
# from torch.utils.data import TensorDataset
# import torch
#
#
# X_test_ndarray = X_test.values
# y_test_ndarray = y_test.values
#
# test_dataset = TensorDataset(torch.from_numpy(X_test_ndarray).type(torch.FloatTensor), torch.from_numpy(y_test_ndarray).type(torch.LongTensor))

In [86]:
# from torch.utils.data import DataLoader
#
#
# test_dataloader = DataLoader(
#     dataset=test_dataset,
#     batch_size=para.batch_size,
#     shuffle=True,
# )

### 构建神经网络

In [87]:
from my_utils.model_class import MLP

model = MLP(in_nums=len(X_train.columns), out_nums=para.classification, drop_p=para.drop)
# to device
model = model.to(device=para.device)
print(model)

MLP(
  (linear_stack): Sequential(
    (0): Linear(in_features=16, out_features=16, bias=True)
    (1): Dropout(p=0.5, inplace=False)
    (2): ReLU()
    (3): Linear(in_features=16, out_features=8, bias=True)
    (4): Dropout(p=0.5, inplace=False)
    (5): ReLU()
    (6): Linear(in_features=8, out_features=2, bias=True)
  )
)


### 训练与测试

In [88]:
import torchmetrics


def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss, correct = 0, 0

    # initialize metric
    train_precision = torchmetrics.Precision(average='none', num_classes=para.classification).to(device=para.device)

    for batch, (X, y) in enumerate(dataloader):

        # to device
        X = X.to(device=para.device)
        y = y.to(device=para.device)

        # compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        train_loss += loss_fn(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        # metric on current batch
        train_precision(pred.argmax(1), y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # if batch % 10 == 0:
        #     loss, current = loss.item(), batch * len(X)
        #     print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    train_loss /= num_batches
    correct /= size
    print(f"Train Error: \n    Accuracy: {(100*correct):>0.1f}%, Avg loss: {train_loss:>8f} \n")

    # metric on all batches using custom accumulation
    total_precision = train_precision.compute()
    print("Precision of every train dataset class: ", total_precision)
    print()

    return correct, train_loss, total_precision


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # initialize metric
    test_precision = torchmetrics.Precision(average='none', num_classes=para.classification).to(device=para.device)

    with torch.no_grad():
        for X, y in dataloader:

            # to device
            X = X.to(device=para.device)
            y = y.to(device=para.device)

            # compute prediction and loss
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

            # metric on current batch
            test_precision(pred.argmax(1), y)


    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n    Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    # metric on all batches using custom accumulation
    total_precision = test_precision.compute()
    print("Precision of every test dataset class: ", total_precision)
    print()

    return correct, test_loss, total_precision

In [89]:
def select_df_to_dataloader(df: pd.DataFrame, select: int) -> DataLoader:

    df = df[df['return_bin'] == select]

    df_dataset = TensorDataset(
        torch.from_numpy(df.loc[:, para.feature_column_start_name: para.feature_column_end_name].values).type(torch.FloatTensor),
        torch.from_numpy(df.loc[:, 'return_bin'].values).type(torch.LongTensor))

    df_dataloader = DataLoader(
        dataset=df_dataset,
        batch_size=para.batch_size,
        shuffle=True,
    )

    return df_dataloader

In [90]:
# temp2_dataloader = select_df_to_dataloader(df=data_cv, select=2)
temp1_dataloader = select_df_to_dataloader(df=data_cv, select=1)
temp0_dataloader = select_df_to_dataloader(df=data_cv, select=0)

In [91]:
from torch import nn
import time
from torch.utils.tensorboard import SummaryWriter

# 计时
time_start = time.time()

# writer = SummaryWriter(para.tensor_board_log_dir)
writer = SummaryWriter()

# 损失函数
loss_fn = nn.CrossEntropyLoss()
# to device
loss_fn = loss_fn.to(device=para.device)

# 优化器
optimizer = torch.optim.Adam(model.parameters(), lr=para.lr)

epochs = para.epochs
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")

    model.train()
    accuracy_train, loss_train, precision_train = train_loop(train_dataloader, model, loss_fn, optimizer)
    model.eval()
    accuracy_cv, loss_cv, precision_cv = test_loop(cv_dataloader, model, loss_fn)

    # accuracy2 = test_loop(temp2_dataloader, model, loss_fn)
    # print('#')
    # accuracy1 = test_loop(temp1_dataloader, model, loss_fn)
    # accuracy0 = test_loop(temp0_dataloader, model, loss_fn)

    # 写入 tensorboard
    if para.classification == 2:

        writer.add_scalars(main_tag=para.info_str+'_evaluation/cv',
                           tag_scalar_dict={
                               'accuracy': accuracy_cv,
                               'precision0': precision_cv[0],
                               'precision1': precision_cv[1]},
                           global_step=t)

        writer.add_scalars(main_tag=para.info_str+'_evaluation/train',
                           tag_scalar_dict={
                               'accuracy': accuracy_train,
                               'precision0': precision_train[0],
                               'precision1': precision_train[1]},
                           global_step=t)

    elif para.classification == 3:

        writer.add_scalars(main_tag=para.info_str+'_evaluation/cv',
                           tag_scalar_dict={
                               'accuracy': accuracy_cv,
                               'precision0': precision_cv[0],
                               'precision1': precision_cv[1],
                               'precision2': precision_cv[2]},
                           global_step=t)

        writer.add_scalars(main_tag=para.info_str+'_evaluation/train',
                           tag_scalar_dict={
                               'accuracy': accuracy_train,
                               'precision0': precision_train[0],
                               'precision1': precision_train[1],
                               'precision2': precision_train[2]},
                           global_step=t)

    elif para.classification == 5:

        writer.add_scalars(main_tag=para.info_str+'_evaluation/cv',
                           tag_scalar_dict={
                               'accuracy': accuracy_cv,
                               'precision0': precision_cv[0],
                               'precision1': precision_cv[1],
                               'precision2': precision_cv[2],
                               'precision3': precision_cv[3],
                               'precision4': precision_cv[4]},
                           global_step=t)

        writer.add_scalars(main_tag=para.info_str+'_evaluation/train',
                           tag_scalar_dict={
                               'accuracy': accuracy_train,
                               'precision0': precision_train[0],
                               'precision1': precision_train[1],
                               'precision2': precision_train[2],
                               'precision3': precision_train[3],
                               'precision4': precision_train[4]},
                           global_step=t)

    writer.add_scalars(main_tag=para.info_str+'_loss/cv',
                       tag_scalar_dict={
                           'loss': loss_cv},
                       global_step=t)

    writer.add_scalars(main_tag=para.info_str+'_loss/train',
                       tag_scalar_dict={
                           'loss': loss_train},
                       global_step=t)
    writer.flush()

    time_end = time.time()
    print('Time cost = %fs' % (time_end - time_start))
    print()

writer.close()

print("Done!")

Epoch 1
-------------------------------
Train Error: 
    Accuracy: 55.2%, Avg loss: 0.686112 

Precision of every train dataset class:  tensor([0.4607, 0.5824], device='cuda:0')

Test Error: 
    Accuracy: 61.7%, Avg loss: 0.686758 

Precision of every test dataset class:  tensor([0.2000, 0.6348], device='cuda:0')

Time cost = 1.158901s

Epoch 2
-------------------------------
Train Error: 
    Accuracy: 56.5%, Avg loss: 0.681254 

Precision of every train dataset class:  tensor([0.4793, 0.5812], device='cuda:0')

Test Error: 
    Accuracy: 65.0%, Avg loss: 0.683742 

Precision of every test dataset class:  tensor([1.0000, 0.6471], device='cuda:0')

Time cost = 2.177240s

Epoch 3
-------------------------------
Train Error: 
    Accuracy: 57.8%, Avg loss: 0.682169 

Precision of every train dataset class:  tensor([0.5361, 0.5824], device='cuda:0')

Test Error: 
    Accuracy: 65.8%, Avg loss: 0.680215 

Precision of every test dataset class:  tensor([0.7500, 0.6552], device='cuda:0')



## 保存模型

In [92]:
torch.save(model.state_dict(), para.save_model_path)

print('Finish save model!')

Finish save model!


## captum

In [93]:
# # captum
# from captum.attr import IntegratedGradients
#
# ig = IntegratedGradients(model)

In [94]:
# temp = cv_dataloader.dataset.tensors[0]
# temp.requires_grad_()
# attr, delta = ig.attribute(temp,target=1, return_convergence_delta=True)
# attr = attr.detach().numpy()

In [95]:
# # Helper method to print importances and visualize distribution
# def visualize_importances(feature_names, importances, title="Average Feature Importances", plot=True, axis_title="Features"):
#     print(title)
#     for i in range(len(feature_names)):
#         print(feature_names[i], ": ", '%.3f'%(importances[i]))
#     y_pos = (np.arange(len(feature_names)))
#     if plot:
#         plt.figure(figsize=(20,6))
#         plt.barh(y_pos, importances, align='center')
#         plt.yticks(y_pos, feature_names)
#         plt.ylabel(axis_title)
#         plt.grid(axis='y')
#         plt.title(title)
# visualize_importances(feature_names=X_cv.columns.values.tolist(), importances=np.mean(attr, axis=0))

In [96]:
# X_cv.columns.values.tolist()

In [97]:
# loss = nn.CrossEntropyLoss()
# input = torch.Tensor(
#     [[-0.0441,  0.0773],
#     [-0.0781, -0.1772],
#     [-0.1319, -0.0432],
#     [-0.0714, -0.1261],
#     [-0.0806, -0.1370],
#     [-0.1730, -0.1472],
#     [-0.0350, -0.0507],
#     [-0.1149, -0.2248]])
# # input = input.reshape(-1,4)
# target = torch.Tensor([0, 1, 1, 0, 0, 0, 0, 0]).type(torch.LongTensor)
# print(input.dtype)
# print(target.dtype)
# output = loss(input, target)
# print(input, target, output)

In [98]:
# # Example of target with class indices
# loss = nn.CrossEntropyLoss()
# input = torch.randn(3, 5, requires_grad=True)
# target = torch.Tensor([1,4,1]).type(torch.LongTensor)
# output = loss(input, target)
# print(input,target,output)

In [99]:
# loss = nn.BCEWithLogitsLoss()
# input = torch.Tensor([0.5, 0.4, 0.3])
# target = torch.Tensor([0])
# output = loss(input, target)
# print(input, target, output)